In [1]:
#divergencias
import util
from backtesting import Backtest
import estrategias as est

#lista = lista= util.lista_de_monedas ()

lista = ['ETHUSDT','BCHUSDT','XRPUSDT','ADAUSDT','FILUSDT','MATICUSDT','OPUSDT','FETUSDT','BTCUSDT'] # lista acotada

#lista=['ETHUSDT']

timeframe = '1h'

###
razon_max = 0.0
razon = 0.0
suma = 0.0
trades = 0
length_sma_max = 0
order_max = 0
length_adx_max = 0
for length_sma in [50]: #range(10,300,20):
    print(f'length_sma: {length_sma}')    
    for order in [550]: #range(200,3000,50):
        print(f'order: {order}')    
        for length_adx in [90]: #range(270,300,10):
            print(f'length_adx: {length_adx}')   
            try:
                suma = 0.0
                trades = 0
                razon = 0.0
                for symbol in lista:
                    try:                
                        #data = est.estrategia_lucho (symbol,timeframe)
                        data = est.estrategia_adx (symbol,timeframe,length_sma,order,length_adx,start_date='2023-09-01')
                        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
                        output = bt.run()
                        if len(lista)==1:
                            bt.plot(filename="graficos/"+symbol)
                        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
                        suma=suma+output['Return [%]']
                        trades=trades+output['# Trades']
                    except Exception as e:
                        print(f"Error en {symbol}: {e}")
                razon = suma/trades
                if razon > razon_max:
                    razon_max = razon
                    length_sma_max = length_sma
                    order_max = order
                    length_adx_max = length_adx
                print(f"Suma total: {suma} - Trades total: {trades} - razón: {razon} - order: {order} - length_adx: {length_adx}")
                print(f"razon_max: {razon_max} - length_sma_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")
            except Exception as e:
                print(f"Error en {symbol}: {e}")
                print(f"Suma total: {suma} - Trades total: {trades} - razón: {razon} - order: {order} - length_adx: {length_adx}")
                print(f"razon_max: {razon_max} - length_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")
print(f"razon_max: {razon_max} - length_sma_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")

length_sma: 50
order: 550
length_adx: 90
umbral: 7.0
Symbol: ETHUSDT - Return [%]: 3.58 - # Trades: 12 - Win Rate [%]: 33.33333333333333
umbral: 7.0
Symbol: BCHUSDT - Return [%]: 5.32 - # Trades: 11 - Win Rate [%]: 36.36363636363637
umbral: 7.0
Symbol: XRPUSDT - Return [%]: 6.58 - # Trades: 11 - Win Rate [%]: 45.45454545454545
umbral: 7.0
Symbol: ADAUSDT - Return [%]: 9.02 - # Trades: 10 - Win Rate [%]: 70.0
umbral: 6.0
Symbol: FILUSDT - Return [%]: 2.48 - # Trades: 7 - Win Rate [%]: 71.42857142857143
umbral: 6.0
Symbol: MATICUSDT - Return [%]: 9.64 - # Trades: 11 - Win Rate [%]: 54.54545454545454
umbral: 6.0
Symbol: OPUSDT - Return [%]: 11.56 - # Trades: 17 - Win Rate [%]: 52.94117647058824
umbral: 5.0
Symbol: FETUSDT - Return [%]: 8.67 - # Trades: 4 - Win Rate [%]: 75.0
umbral: 7.0
Symbol: BTCUSDT - Return [%]: 16.81 - # Trades: 14 - Win Rate [%]: 42.857142857142854
Suma total: 73.66094386200123 - Trades total: 97 - razón: 0.7593911738350643 - order: 550 - length_adx: 90
razon_max: 0

In [2]:
data

,Open Time,Open,High,Low,Close,Volume,atr,Indicator1,Indicator2,adx,pico_inferior,adx_condicion,trade,stop_loss,take_profit,porcentajeentrada,cerrar
timestamp,,,,,,,,,,,,,,,,,
2024-10-01 00:00:00,2024-10-01 00:00:00,63309.0,63428.0,63305.0,63423.7,575.602,NaN,None,NaN,NaN,NaN,False,-1.5,0.0,0.0,0.99,False
2024-10-01 00:01:00,2024-10-01 00:01:00,63423.7,63491.1,63380.8,63380.8,640.452,NaN,None,NaN,NaN,NaN,False,-1.5,0.0,0.0,0.99,False
2024-10-01 00:02:00,2024-10-01 00:02:00,63380.8,63380.8,63290.8,63327.2,450.107,NaN,None,NaN,NaN,NaN,False,-1.5,0.0,0.0,0.99,False
2024-10-01 00:03:00,2024-10-01 00:03:00,63327.2,63330.3,63260.1,63317.3,346.579,NaN,None,NaN,NaN,NaN,False,-1.5,0.0,0.0,0.99,False
2024-10-01 00:04:00,2024-10-01 00:04:00,63317.2,63355.0,63315.2,63315.2,160.253,NaN,None,NaN,NaN,NaN,False,-1.5,0.0,0.0,0.99,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-16 20:46:00,2024-10-16 20:46:00,67871.6,67881.3,67859.8,67881.3,139.123,28.361997,None,67826.392,4.141828,NaN,False,-1.5,0.0,0.0,0.99,False
2024-10-16 20:47:00,2024-10-16 20:47:00,67881.3,67923.6,67881.2,67923.6,168.018,29.364712,None,67829.804,4.146337,NaN,False,-1.5,0.0,0.0,0.99,False
2024-10-16 20:48:00,2024-10-16 20:48:00,67923.5,67945.0,67885.2,67900.0,135.331,31.538661,None,67832.694,4.162569,NaN,False,-1.5,0.0,0.0,0.99,False
